# movie data preprocessing
## 프로젝트에 사용되지 않는 컬럼을 버린다
## movies와 credits dataframe을 merge한다.
### budget, homepage, original_language, original_title, production_countries, release_date, revenue, spoken_language, status
## crew 컬럼에서 director를 뽑아내서 새로운 컬럼으로 만든다.
### director1 ~ director 종류수

In [1]:
import pandas as pd
import numpy as npc

movies_df = pd.read_csv("data/tmdb_5000_movies.csv")
# drop columns
# budget, homepage, original_language, original_title, production_countries, release_date,
# revenue, spoken_languages, status
movies_df = movies_df.drop(columns=['budget', 'homepage', 'original_language', 'original_title', 'production_countries',
                                    'release_date', 'revenue', 'spoken_languages', 'status'])

In [2]:
credits_df = pd.read_csv("data/tmdb_5000_credits.csv")
credits_df = credits_df.drop(columns=['title'])
credits_df.columns=['id', 'cast', 'crew']
credits_df.head()

,id,cast,crew
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [3]:
total_df = movies_df.join(credits_df.set_index('id'), on='id')

In [4]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 13 columns):
genres                  4803 non-null object
id                      4803 non-null int64
keywords                4803 non-null object
overview                4800 non-null object
popularity              4803 non-null float64
production_companies    4803 non-null object
runtime                 4801 non-null float64
tagline                 3959 non-null object
title                   4803 non-null object
vote_average            4803 non-null float64
vote_count              4803 non-null int64
cast                    4803 non-null object
crew                    4803 non-null object
dtypes: float64(3), int64(2), object(8)
memory usage: 487.9+ KB


In [5]:
import json

json_columns = ['genres', 'keywords', 'cast', 'crew', 'production_companies']
for column in json_columns:
    total_df[column] = total_df[column].apply(json.loads)


In [6]:
count=0
for title, cast_list in zip(total_df['title'], total_df["crew"]):
    if not cast_list:
        print(title)
        count+=1
print(count)

Flying By
Running Forever
Boynton Beach Club
Sharkskin
The Book of Mormon Movie, Volume 1: The Journey
Hum To Mohabbat Karega
Roadside Romeo
Me You and Five Bucks
Down & Out With The Dolls
Crowsnest
Sex With Strangers
The Helix... Loaded
Karachi se Lahore
Harrison Montgomery
Light from the Darkroom
America Is Still the Place
The Little Ponderosa Zoo
Diamond Ruff
Rise of the Entrepreneur: The Search for a Better Way
I Want Your Money
Fabled
Death Calls
Amidst the Devil's Wings
Teeth and Blood
UnDivided
Little Big Top
Short Cut to Nirvana: Kumbh Mela
The Blood of My Brother: A Story of Death in Iraq
28


In [7]:
director_list = list()
director_set = set()
for title, crew_list in zip(total_df['title'], total_df['crew']):
    temp = list()
    for one_crew_dict in crew_list:
        if one_crew_dict['job'] == 'Director':
            temp.append(one_crew_dict['name'])
            director_set.add(one_crew_dict['name'])
    director_list.append(temp)
#total_df['director'] = directors

In [8]:
director_list
director_set
director_dict = dict()
director_dict = dict.fromkeys(director_set)
count = 0
for key in director_dict.keys():
    director_dict[key] = count
    count+=1
director_rev_dict = {v:k for k,v in director_dict.items()}
print(count)

2577


In [9]:
director_vector = list()
for directors in director_list:
    temp_director = [0]*2577
    temp_name = list()
    for director in directors:
        temp_director[director_dict[director]] = 1
    director_vector.append(temp_director)

print(len(director_vector))


4803


In [10]:
import numpy as np

column_names = list()
count = 0
while True:
    if count >= 2577:
        break
    name = 'Director' + str(count)
    column_names.append(name)
    count += 1
Directors = pd.DataFrame(director_vector, columns=column_names)


In [11]:
Directors['names'] = director_list

In [12]:
director_count = list()
for item in director_list:
    director_count.append(len(item))
Directors['count'] = director_count

In [13]:
Directors['title'] = total_df['title']

In [14]:
title_director = pd.DataFrame({'title' : Directors['title'], 'count' : Directors['count']})

In [15]:
title_director.head(10)

,title,count
0,Avatar,1
1,Pirates of the Caribbean: At World's End,1
2,Spectre,1
3,The Dark Knight Rises,1
4,John Carter,1
5,Spider-Man 3,1
6,Tangled,2
7,Avengers: Age of Ultron,1
8,Harry Potter and the Half-Blood Prince,1
9,Batman v Superman: Dawn of Justice,1


In [16]:
title_director.plot(x='title', y='count')

# Production Companies vectorize
## production company 의 종류 개수 : 5017
## 한 영화가 가진 가장 많은 production company의 종류 개수 : 26
## 한 영화가 가진 가장 적은 production company의 종류 개수 : 0

In [17]:
pro_com_set = set()
com_list_list = list()
for company_list in total_df['production_companies']:
    temp_com_list = list()
    for company in company_list:
        pro_com_set.add(company['name'])
        temp_com_list.append(company['name'])
    com_list_list.append(temp_com_list)

pro_com_vectors = list()
pro_com_dict = dict()
pro_com_dict = dict.fromkeys(pro_com_set)
count = 0
for keyitem in pro_com_dict.keys():
    pro_com_dict[keyitem] = count
    count += 1

for com_list in com_list_list:
    temp_vec = [0]*5017
    for com in com_list:
        temp_vec[pro_com_dict[com]] = 1
    pro_com_vectors.append(temp_vec)

production_com_columns = list()
for i in range(5017):
    temp_name = 'company' + str(i)
    production_com_columns.append(temp_name)

Production_Companies = pd.DataFrame(pro_com_vectors, columns = production_com_columns)
Production_Companies['title'] = total_df['title']

In [18]:
# 한 영화가 가진 가장 많은 production company의 종류 개수 : 26
max_len = 0
for com_list in com_list_list:
    if len(com_list) > max_len:
        max_len = len(com_list)
print(max_len)

26


In [19]:
# 한 영화가 가진 가장 적은 production company의 종류 개수 : 0
min_len = 9999
for com_list in com_list_list:
    if len(com_list) < min_len:
        min_len = len(com_list)
print(min_len)

0


# Genre vectorize
## genre 의 종류 개수 : 20
## 한 영화가 가진 가장 많은 genre의 종류 개수 : 7
## 한 영화가 가진 가장 적은 genre의 종류 개수 : 0

In [20]:
genre_set = set()
genre_list_list = list()

for genre_list in total_df['genres']:
    temp_genre = list()
    for genre in genre_list:
        genre_set.add(genre['name'])
        temp_genre.append(genre['name'])
    genre_list_list.append(temp_genre)
print(len(genre_set))
print(genre_set)

20
{'History', 'War', 'Adventure', 'Horror', 'Science Fiction', 'Documentary', 'TV Movie', 'Action', 'Family', 'Western', 'Comedy', 'Foreign', 'Romance', 'Mystery', 'Music', 'Thriller', 'Animation', 'Drama', 'Fantasy', 'Crime'}


In [21]:
genre_dict = dict()
genre_dict = dict.fromkeys(genre_set)
count = 0
for key_item in genre_dict.keys():
    genre_dict[key_item] = count
    count += 1
genre_vector = list()
for genre_list in genre_list_list:
    temp_genre_vec = [0] * 20
    for genre in genre_list:
        temp_genre_vec[genre_dict[genre]] = 1
    genre_vector.append(temp_genre_vec)
genre_cols = list()
for i in range(20):
    temp_genre = 'genre' + str(i)
    genre_cols.append(temp_genre)

Genre = pd.DataFrame(genre_vector, columns=genre_cols)

In [22]:
Genre.head(10)

,genre0,genre1,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,genre10,genre11,genre12,genre13,genre14,genre15,genre16,genre17,genre18,genre19
0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1
4,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
7,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
9,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0


In [23]:
# 한 영화가 가진 가장 많은 genre의 종류 개수 : 7
max_len = 0
for genre_list in genre_list_list:
    if len(genre_list) > max_len:
        max_len = len(genre_list)
print(max_len)

7


In [24]:
# 한 영화가 가진 가장 적은 genre의 종류 개수 : 0
min_len = 9999
for genre_list in genre_list_list:
    if len(genre_list) < min_len:
        min_len = len(genre_list)
print(min_len)

0
